In [4]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import metrics
from sklearn import preprocessing
from sklearn.preprocessing import label_binarize
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import log_loss
from sklearn.naive_bayes import BernoulliNB
from matplotlib import style
import matplotlib.pyplot as pl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
%matplotlib inline

In [5]:
#reading in the test and target data
# training and test must match column wise
train=pd.read_csv('/Users/sawada/Dropbox/fall2016/cs4661/hw/final-project/weather/weather_1.csv',parse_dates = ['Dates'])
test=pd.read_csv("/Users/sawada/Dropbox/fall2016/cs4661/hw/final-project/test.csv",parse_dates = ['Dates'])
# test.head()

# train.isnull().sum()

In [6]:
# TRAINING data
#Convert crime labels to numbers
df_crime = preprocessing.LabelEncoder()
crime = df_crime.fit_transform(train.Category)
#Get binarized resolution, weatherDesc, weekdays, districts, and hours using dummy variables
days = pd.get_dummies(train.DayOfWeek)

district = pd.get_dummies(train.PdDistrict)
resolution = pd.get_dummies(train.Resolution)

hour = train.Dates.dt.hour
hour = pd.get_dummies(hour)
#Build new array
train_data = pd.concat([hour, days, district, resolution], axis=1)
train_data['crime']=crime

train_data.columns.values
# days.head()

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 'Friday', 'Monday', 'Saturday', 'Sunday',
       'Tuesday', 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE',
       'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL',
       'TENDERLOIN', 'ARREST, BOOKED', 'ARREST, CITED',
       'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'EXCEPTIONAL CLEARANCE',
       'JUVENILE BOOKED', 'LOCATED', 'NONE', 'PSYCHOPATHIC CASE',
       'UNFOUNDED', 'crime'], dtype=object)

In [10]:
features = ['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday',
 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION',
 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']

print(features)
 
features2 = [x for x in range(0,24)]
features = train_data.columns.values.tolist()
    
training, testing = train_test_split(train_data, train_size=.50) 

['Friday', 'Monday', 'Saturday', 'Sunday', 'Thursday', 'Tuesday', 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN']


## log_loss

In [11]:
#For this competition the metric used to rate the performance of the model
#is the multi-class log_loss — smaller values of this loss correspond to improved performance.
#Logistic Regression for comparison closer to 0 the better , cant use accuracy for regressions
model_lr = LogisticRegression()
model_lr.fit(training[features], training['crime'])
predicted1 = np.array(model_lr.predict_proba(testing[features]))
print(log_loss(testing['crime'], predicted1))

2.16557043533


In [12]:
print(predicted1)

[[  6.51228027e-13   9.43566112e-11   3.16778666e-02 ...,   3.64226972e-02
    3.46568412e-03   7.79914684e-03]
 [  7.71174434e-13   4.86205457e-11   2.74172944e-02 ...,   2.83805480e-02
    2.77222891e-03   4.38729681e-03]
 [  8.47562904e-13   1.01002198e-10   2.05351511e-02 ...,   2.08115551e-02
    2.61602946e-03   4.46362154e-03]
 ..., 
 [  3.52697297e-27   5.40186428e-25   3.20263812e-05 ...,   2.40910449e-01
    2.71662262e-02   1.35266406e-02]
 [  6.06983056e-16   1.37594195e-13   7.52258453e-03 ...,   1.88949641e-02
    8.89563638e-03   1.24110263e-02]
 [  3.48274753e-11   7.90864906e-09   7.20814428e-02 ...,   3.46712611e-03
    1.75447820e-03   2.30534750e-03]]


### KNN

In [56]:
print(training.columns.values.tolist())
train_X =  train_data[training.columns.values.tolist()]
train_y = train_data['crime']

print(train_y.shape, train_X.shape)
# split data into training and testing set
X_train, X_test, y_train, y_test = train_test_split(train_X, train_y, test_size=0.4, random_state=1)
    
    

knn = KNeighborsClassifier( n_neighbors = 10 ) 
knn.fit( train_X, train_y )
y_predict = knn.predict(X_test)
score = accuracy_score(y_test, y_predict)
    
print('Accuracy Score: ', score)
#     accuracies.append(score)
    
# for i, a in enumerate(accuracies):
#     print('k = ' + str(k_list[i]) + ', Accuracy Score: ' + str(acc[i])) #Printing the scores

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 'Friday', 'Monday', 'Saturday', 'Sunday', 'Tuesday', 'Wednesday', 'BAYVIEW', 'CENTRAL', 'INGLESIDE', 'MISSION', 'NORTHERN', 'PARK', 'RICHMOND', 'SOUTHERN', 'TARAVAL', 'TENDERLOIN', 'ARREST, BOOKED', 'ARREST, CITED', 'CLEARED-CONTACT JUVENILE FOR MORE INFO', 'EXCEPTIONAL CLEARANCE', 'JUVENILE BOOKED', 'LOCATED', 'NONE', 'PSYCHOPATHIC CASE', 'UNFOUNDED', 'crime']
(2096,) (2096, 50)
Accuracy Score:  0.898688915375


### Iterate through various k-values to train the model.
_Note_: Instantiate a KNN object with different k value in each iteration.

In [55]:
k_list = [1,5,10,20,40,80,150,200]
accuracies = []

for k in k_list:
    knn = KNeighborsClassifier( n_neighbors = k )
    knn.fit( X_train, y_train )
    y_predict = knn.predict(X_test)
    score = accuracy_score(y_test, y_predict)
    
    accuracies.append(score)
    
for i, a in enumerate(accuracies):
        print('k = ' + str(k_list[i]) + ', Accuracy Score: ' + str(accuracies[i])) 

k = 1, Accuracy Score: 0.878426698451
k = 5, Accuracy Score: 0.880810488677
k = 10, Accuracy Score: 0.908224076281
k = 20, Accuracy Score: 0.902264600715
k = 40, Accuracy Score: 0.809296781883
k = 80, Accuracy Score: 0.81168057211
k = 150, Accuracy Score: 0.631704410012
k = 200, Accuracy Score: 0.624553039333


## Naive Bayes

In [13]:
#bernoulliNB
# predicting only on the training data
model_B = BernoulliNB()
model_B.fit(training[features], training['crime'])
predicted2 = np.array(model_B.predict_proba(testing[features]))
log_loss(testing['crime'], predicted2)

4.0107469470477701